In [1]:
from pathlib import Path
import re

# Load Frankenstein
# text = Path("frankenstein.txt").read_text()
text = Path("frankenstein.txt").read_text(encoding="utf-8")


# Clean & split into chunks
# def chunk_text(text, max_words=350):
#     words = text.split()
#     chunks = []
#     for i in range(0, len(words), max_words):
#         chunk = " ".join(words[i:i+max_words])
#         chunks.append(chunk)
#     return chunks

def chunk_text(text, size=800, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + size
        chunks.append(text[start:end])
        start += size - overlap
    return chunks


chunks = chunk_text(text)
len(chunks)


699

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

chunk_embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
chunk_embeddings.shape


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

(699, 384)

In [3]:
import faiss

dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(chunk_embeddings)


In [4]:
import faiss

dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(chunk_embeddings)


In [5]:
from huggingface_hub import notebook_login
notebook_login()


In [6]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '691de23843ce34b543fd62e2', 'name': 'Thejass', 'fullname': 'Thejaswin', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/noauth/oq5hRpUZgGxbvmHnSGgs1.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'frankensteinChatbot', 'role': 'fineGrained', 'createdAt': '2025-11-20T22:24:29.102Z', 'fineGrained': {'canReadGatedRepos': True, 'global': [], 'scoped': [{'entity': {'_id': '669650bb11dbbf600cf4dcf0', 'type': 'model', 'name': 'google/gemma-2-2b-it'}, 'permissions': ['repo.content.read']}, {'entity': {'_id': '691de23843ce34b543fd62e2', 'type': 'user', 'name': 'Thejass'}, 'permissions': ['repo.content.read']}]}}}}


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-2-2b-it"
# model_name = "google/gemma-1.1-2b-it" # Access Denied
# model_name = "mlabonne/opengemma-2b-it" # Access Denied
# model_name = "google/gemma-2b-it" # Access Denied
# model_name = "google/gemma-3-4b-it" # Access Denied
# model_name = "google/gemma-3-4b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,    # force CPU
    device_map="cpu"              # ensure CPU-only
)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def retrieve(query, k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(q_emb, k)
    return [chunks[i] for i in indices[0]]


In [9]:
def generate_answer(query):
    # Retrieve relevant context
    retrieved_chunks = retrieve(query, k=3)
    context = "\n\n".join(retrieved_chunks)

    prompt = (
        "You are a helpful assistant answering questions about Mary Shelley's *Frankenstein*.\n"
        "Use ONLY the following context:\n\n"
        f"{context}\n\n"
        "Question: " + query + "\nAnswer:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    output = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.4
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [11]:
def chat():
    print("Frankenstein Chatbot — type 'exit' to quit\n")

    while True:
        user = input("You: ")
        if user.lower() in {"exit", "quit"}:
            break

        answer = generate_answer(user)
        print("\nBot:", answer, "\n")

chat()

Frankenstein Chatbot — type 'exit' to quit


Bot: You are a helpful assistant answering questions about Mary Shelley's *Frankenstein*.
Use ONLY the following context:

oying his
enemy, were now suspended by a mixture of curiosity and compassion. I
approached this tremendous being; I dared not again raise my eyes to his
face, there was something so scaring and unearthly in his ugliness. I
attempted to speak, but the words died away on my lips. The monster
continued to utter wild and incoherent self-reproaches. At length I
gathered resolution to address him in a pause of the tempest of his passion.

“Your repentance,” I said, “is now superfluous. If you
had listened to the voice of conscience and heeded the stings of remorse
before you had urged your diabolical vengeance to this extremity,
Frankenstein would yet have lived.”

“And do you dream?” said the dæmon. “Do you think that I was then
dead to agony and remorse? He,” he continued, pointing to the corp

her of their
charge was one of